In [1]:
from sklearn.metrics import confusion_matrix as cnfm
import itertools
import math
from sklearn.utils import shuffle
!pip install wandb
import wandb

wandb.init(project='Assignment 1', entity='CS23M028')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.5/263.5 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.0 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist
import copy

In [3]:
def normalize_data(x):
  x_norm = x.astype('float32')
  x_norm = x_norm / 255.0
  return x_norm

In [4]:
def func(activation,a_k1):
  a_k = np.clip(a_k1, -55, 55)
  if(activation == "tanh"):
    a_k = np.tanh(a_k)
  elif(activation == "sigmoid"):
    a_k = 1/(1 + np.exp(-1*a_k))
  else:
    a_k = np.maximum(0,a_k)
  return a_k

In [5]:
def derivativeFun(activation,a_k1):
  a_k = np.clip(a_k1, -55, 55)
  activationResult = func(activation,a_k)
  if(activation == "tanh"):
    activationResult = 1 - (activationResult**2)
  elif(activation == "sigmoid"):
    activationResult = activationResult - (activationResult**2)
  else:
    activationResult = np.where(a_k > 0, 1, 0)
  return activationResult

In [6]:
def decision(a_k,classificationFunction):
  if classificationFunction == "crossEntropy":
    a_k = np.exp(a_k - np.max(a_k))
    a_k = a_k / sum(a_k)
  return a_k

In [7]:
def forwardProp(inputX,activation,classificationFunction,weights,bias):
  h_k = inputX
  PreActivations = list()
  PostActivations = list()
  PostActivations.append(h_k)
  layers = len(weights) - 1
  for k in range(0,layers):
    a_k = bias[k] + np.dot(weights[k],h_k)
    PreActivations.append(a_k)
    h_k = func(activation,a_k)
    PostActivations.append(h_k)
  a_k = bias[layers] + np.matmul(weights[layers],h_k)
  PreActivations.append(a_k)
  yPred = decision(a_k,classificationFunction)
  return PreActivations,PostActivations,yPred

In [8]:
def backProp(real, pred, h_k, weights, activation, PreActivations):
    a_l_L_theta = pred - real
    currentActivationGradient = a_l_L_theta
    WeightGradients = []
    biasGradients = []
    layers = len(weights) - 1

    for i in range(layers, -1, -1):
        W_i_L_theta = currentActivationGradient*np.transpose(h_k[i])
        WeightGradients.insert(0, W_i_L_theta)
        b_i_L_theta = np.sum(currentActivationGradient, axis=0, keepdims=True)
        biasGradients.insert(0, b_i_L_theta)

        if i > 0:
            h_i_prev_L_theta = np.matmul(weights[i].T, currentActivationGradient)
            currentActivationGradient = h_i_prev_L_theta * derivativeFun(activation, PreActivations[i - 1])

    return WeightGradients, biasGradients

In [9]:
def randomizer(dim1,dim2,init_weight):
  std_dev = 0.1
  if(init_weight == "xavier"):
    variance = 2.0 / (dim1 + dim2)
    std_dev = np.sqrt(variance)
  return std_dev

In [10]:
def stochastic_gradient_descent(nodesPerLayer, x_flatten_train, y_encoded, batch_size,activationFunc,epochs,lr,x_flatten_test,y_flatten_test,x_val,y_val,init_weight,lambda_reg):
    weights = [np.random.randn(nodesPerLayer[i], nodesPerLayer[i-1]) * randomizer(nodesPerLayer[i], nodesPerLayer[i-1],init_weight) for i in range(1, len(nodesPerLayer))]
    bias = [np.random.randn(nodesPerLayer[i], 1) * 0.1 for i in range(1, len(nodesPerLayer))]

    num_batches = len(x_flatten_train)

    for epoch in range(epochs):
        for batch in range(0,num_batches):
            if(batch % (num_batches/10) == 0):
              print("batch: " + str(batch))
            start = batch * batch_size
            end = (batch + 1) * batch_size

            batch_x = x_flatten_train[start:end]
            batch_y = y_encoded[start:end]

            batch_Wdelta = [np.zeros_like(w) for w in weights]
            batch_Bdelta = [np.zeros_like(b) for b in bias]

            for j in range(len(batch_x)):
                A, B, C = forwardProp(batch_x[j], activationFunc, "crossEntropy", weights, bias)
                Wdelta, Bdelta = backProp(batch_y[j], C, B, weights, activationFunc, A)

                for k in range(len(batch_Wdelta)):
                    batch_Wdelta[k] += Wdelta[k]
                    batch_Bdelta[k] += Bdelta[k]

            for k in range(len(weights)):
                weights[k] = ((1 - (lr * lambda_reg / batch_size)) * weights[k]) - lr * (batch_Wdelta[k] / batch_size)
                bias[k] -= lr * (batch_Bdelta[k] / batch_size)
        accuracy,loss = testModel(weights,bias,x_flatten_test,y_flatten_test,activationFunc)
        Valaccuracy,Valloss = testModel(weights,bias,x_val,y_val,activationFunc)
        print({"val_loss":Valloss,"val_accuracy":Valaccuracy,"loss":loss,"accuracy":accuracy,"epoch":epoch})
        wandb.log({"val_loss":Valloss,"val_accuracy":Valaccuracy,"loss":loss,"accuracy":accuracy,"epoch":epoch})

    return weights, bias


In [11]:
def momentum_gradient_descent(nodesPerLayer, x_flatten_train, y_encoded, gamma, batch_size,activationFunc,epochs,lr,x_flatten_test,y_flatten_test,x_val,y_val,init_weight,lambda_reg):

    weights = [np.random.randn(nodesPerLayer[i], nodesPerLayer[i-1]) * randomizer(nodesPerLayer[i], nodesPerLayer[i-1],init_weight) for i in range(1, len(nodesPerLayer))]
    bias = [np.random.randn(nodesPerLayer[i], 1) * 0.1 for i in range(1, len(nodesPerLayer))]

    Wdelta = [np.zeros((nodesPerLayer[i], nodesPerLayer[i-1])) for i in range(1, len(nodesPerLayer))]
    Bdelta = [np.zeros((nodesPerLayer[i], 1)) for i in range(1, len(nodesPerLayer))]

    num_batches = len(x_flatten_train)

    for epoch in range(epochs):
        for batch in range(0,num_batches):
            if(batch % (num_batches/10) == 0):
              print("batch :" + str(batch))
            start = batch * batch_size
            end = (batch + 1) * batch_size

            batch_x = x_flatten_train[start:end]
            batch_y = y_encoded[start:end]

            batch_Wdelta = [np.zeros_like(w) for w in weights]
            batch_Bdelta = [np.zeros_like(b) for b in bias]

            for j in range(len(batch_x)):
                A, B, C = forwardProp(batch_x[j], activationFunc, "crossEntropy", weights, bias)
                CurrWdelta, CurrBdelta = backProp(batch_y[j], C, B, weights,activationFunc, A)

                for k in range(len(batch_Wdelta)):
                    batch_Wdelta[k] += CurrWdelta[k]
                    batch_Bdelta[k] += CurrBdelta[k]

            for k in range(len(weights)):
                Wdelta[k] = gamma * Wdelta[k] + lr * batch_Wdelta[k] / batch_size
                Bdelta[k] = gamma * Bdelta[k] + lr * batch_Bdelta[k] / batch_size

                weights[k] = ((1 - (lr * lambda_reg / batch_size)) * weights[k]) -Wdelta[k]
                bias[k] -= Bdelta[k]
        accuracy,loss = testModel(weights,bias,x_flatten_test,y_flatten_test,activationFunc)
        Valaccuracy,Valloss = testModel(weights,bias,x_val,y_val,activationFunc)
        print({"val_loss":Valloss,"val_accuracy":Valaccuracy,"loss":loss,"accuracy":accuracy,"epoch":epoch})
        wandb.log({"val_loss":Valloss,"val_accuracy":Valaccuracy,"loss":loss,"accuracy":accuracy,"epoch":epoch})

    return weights, bias


In [12]:
def nesterov_gradient_descent(nodesPerLayer,x_flatten_train,y_encoded,gamma, batch_size,activationFunc,epochs,lr,x_flatten_test,y_flatten_test,x_val,y_val,init_weight,lambda_reg):
    weights = [np.random.randn(nodesPerLayer[i], nodesPerLayer[i-1]) * randomizer(nodesPerLayer[i], nodesPerLayer[i-1],init_weight) for i in range(1, len(nodesPerLayer))]
    bias = [np.random.randn(nodesPerLayer[i], 1) * 0.1 for i in range(1, len(nodesPerLayer))]

    num_batches = len(x_flatten_train)

    for epoch in range(epochs):
        for batch in range(0,num_batches):
            start = batch * batch_size
            end = (batch + 1) * batch_size

            batch_x = x_flatten_train[start:end]
            batch_y = y_encoded[start:end]

            lookahead_weights = [w - gamma * dw for w, dw in zip(weights, weights)]
            lookahead_bias = [b - gamma * db for b, db in zip(bias, bias)]

            for j in range(len(batch_x)):
                A, B, C = forwardProp(batch_x[j], activationFunc, "crossEntropy", lookahead_weights, lookahead_bias)
                CurrWdelta, CurrBdelta = backProp(batch_y[j], C, B, lookahead_weights, activationFunc, A)

                for k in range(len(weights)):
                    weights[k] = ((1 - (lr * lambda_reg / batch_size)) * weights[k]) - lr * CurrWdelta[k]
                    bias[k] -= lr * CurrBdelta[k]
        accuracy,loss = testModel(weights,bias,x_flatten_test,y_flatten_test,activationFunc)
        Valaccuracy,Valloss = testModel(weights,bias,x_val,y_val,activationFunc)
        print({"val_loss":Valloss,"val_accuracy":Valaccuracy,"loss":loss,"accuracy":accuracy,"epoch":epoch})
        wandb.log({"val_loss":Valloss,"val_accuracy":Valaccuracy,"loss":loss,"accuracy":accuracy,"epoch":epoch})
    return weights, bias


In [13]:
def rmsprop(nodesPerLayer, x_flatten_train, y_encoded, beta, eps, epochs, batch_size, lr,activationFunc,x_flatten_test,y_flatten_test,x_val,y_val,init_weight,lambda_reg):
    weights = [np.random.randn(nodesPerLayer[i], nodesPerLayer[i-1]) * randomizer(nodesPerLayer[i], nodesPerLayer[i-1],init_weight) for i in range(1, len(nodesPerLayer))]
    bias = [np.random.randn(nodesPerLayer[i], 1) * 0.1 for i in range(1, len(nodesPerLayer))]

    rmsweights = [np.zeros((nodesPerLayer[i], nodesPerLayer[i-1])) for i in range(1, len(nodesPerLayer))]
    rmsbias = [np.zeros((nodesPerLayer[i], 1)) for i in range(1, len(nodesPerLayer))]

    num_batches = len(x_flatten_train)

    for epoch in range(epochs):
        for batch in range(0,num_batches):
            start = batch * batch_size
            end = (batch + 1) * batch_size

            batch_x = x_flatten_train[start:end]
            batch_y = y_encoded[start:end]

            batch_w_delta = [np.zeros_like(w) for w in weights]
            batch_b_delta = [np.zeros_like(b) for b in bias]

            for j in range(len(batch_x)):
                A, B, C = forwardProp(batch_x[j], activationFunc, "crossEntropy", weights, bias)
                CurrWdelta, CurrBdelta = backProp(batch_y[j], C, B, weights, activationFunc, A)

                for k in range(len(CurrWdelta)):
                    batch_w_delta[k] += CurrWdelta[k]
                    batch_b_delta[k] += CurrBdelta[k]

            for k in range(len(batch_w_delta)):
                rmsweights[k] = beta * rmsweights[k] + (1 - beta) * (batch_w_delta[k] ** 2)
                rmsbias[k] = beta * rmsbias[k] + (1 - beta) * (batch_b_delta[k] ** 2)

                weights[k] =((1 - (lr * lambda_reg / batch_size)) * weights[k]) -( (lr * batch_w_delta[k]) / (np.sqrt(rmsweights[k]) + eps))
                bias[k] -= (lr * batch_b_delta[k]) / (np.sqrt(rmsbias[k]) + eps)
        accuracy,loss = testModel(weights,bias,x_flatten_test,y_flatten_test,activationFunc)
        Valaccuracy,Valloss = testModel(weights,bias,x_val,y_val,activationFunc)
        print({"val_loss":Valloss,"val_accuracy":Valaccuracy,"loss":loss,"accuracy":accuracy,"epoch":epoch})
        wandb.log({"val_loss":Valloss,"val_accuracy":Valaccuracy,"loss":loss,"accuracy":accuracy,"epoch":epoch})

    return weights, bias

In [14]:
def adam(nodesPerLayer, x_flatten_train, y_encoded, beta1, beta2, eps, batch_size, lr,activationFunc,epochs,x_flatten_test,y_flatten_test,x_val,y_val,init_weight,lambda_reg):
    # Initialize weights and biases
    weights = [np.random.randn(nodesPerLayer[i], nodesPerLayer[i-1]) * randomizer(nodesPerLayer[i], nodesPerLayer[i-1],init_weight) for i in range(1, len(nodesPerLayer))]
    bias = [np.random.randn(nodesPerLayer[i], 1) * 0.1 for i in range(1, len(nodesPerLayer))]

    # Initialize Adam parameters
    m_weights = [np.zeros((nodesPerLayer[i], nodesPerLayer[i-1])) for i in range(1, len(nodesPerLayer))]
    v_weights = [np.zeros((nodesPerLayer[i], nodesPerLayer[i-1])) for i in range(1, len(nodesPerLayer))]
    m_bias = [np.zeros((nodesPerLayer[i], 1)) for i in range(1, len(nodesPerLayer))]
    v_bias = [np.zeros((nodesPerLayer[i], 1)) for i in range(1, len(nodesPerLayer))]

    num_batches = len(x_flatten_train)

    for epoch in range(epochs):
        for batch in range(0,num_batches):
            start = batch * batch_size
            end = (batch + 1) * batch_size

            batch_x = x_flatten_train[start:end]
            batch_y = y_encoded[start:end]

            batch_w_delta = [np.zeros_like(w) for w in weights]
            batch_b_delta = [np.zeros_like(b) for b in bias]

            for j in range(len(batch_x)):
                A, B, C = forwardProp(batch_x[j], activationFunc, "crossEntropy", weights, bias)
                CurrWdelta, CurrBdelta = backProp(batch_y[j], C, B, weights, activationFunc, A)

                for k in range(len(CurrWdelta)):
                    batch_w_delta[k] += CurrWdelta[k]
                    batch_b_delta[k] += CurrBdelta[k]

            for k in range(len(batch_w_delta)):
                m_weights[k] = beta1 * m_weights[k] + (1 - beta1) * batch_w_delta[k]
                v_weights[k] = beta2 * v_weights[k] + (1 - beta2) * (batch_w_delta[k] ** 2)
                m_bias[k] = beta1 * m_bias[k] + (1 - beta1) * batch_b_delta[k]
                v_bias[k] = beta2 * v_bias[k] + (1 - beta2) * (batch_b_delta[k] ** 2)

                m_weights_hat = m_weights[k] / (1 - beta1 ** (epoch + 1))
                v_weights_hat = v_weights[k] / (1 - beta2 ** (epoch + 1))
                m_bias_hat = m_bias[k] / (1 - beta1 ** (epoch + 1))
                v_bias_hat = v_bias[k] / (1 - beta2 ** (epoch + 1))

                weights[k] = ((1 - (lr * lambda_reg / batch_size)) * weights[k]) - ( (lr * m_weights_hat) / (np.sqrt(v_weights_hat) + eps))
                bias[k] -= (lr * m_bias_hat) / (np.sqrt(v_bias_hat) + eps)

        accuracy,loss = testModel(weights,bias,x_flatten_test,y_flatten_test,activationFunc)
        Valaccuracy,Valloss = testModel(weights,bias,x_val,y_val,activationFunc)
        print({"val_loss":Valloss,"val_accuracy":Valaccuracy,"loss":loss,"accuracy":accuracy,"epoch":epoch})
        wandb.log({"val_loss":Valloss,"val_accuracy":Valaccuracy,"loss":loss,"accuracy":accuracy,"epoch":epoch})


    return weights, bias


In [15]:
def nadam(nodesPerLayer, x_flatten_train, y_encoded, beta1, beta2, eps, batch_size, lr,activationFunc,epochs,x_flatten_test,y_flatten_test,x_val,y_val,init_weight,lambda_reg):
    # Initialize weights and biases
    weights = [np.random.randn(nodesPerLayer[i], nodesPerLayer[i-1]) * randomizer(nodesPerLayer[i], nodesPerLayer[i-1],init_weight) for i in range(1, len(nodesPerLayer))]
    bias = [np.random.randn(nodesPerLayer[i], 1) * 0.1 for i in range(1, len(nodesPerLayer))]

    # Initialize Nadam parameters
    m_weights = [np.zeros((nodesPerLayer[i], nodesPerLayer[i-1])) for i in range(1, len(nodesPerLayer))]
    v_weights = [np.zeros((nodesPerLayer[i], nodesPerLayer[i-1])) for i in range(1, len(nodesPerLayer))]
    m_bias = [np.zeros((nodesPerLayer[i], 1)) for i in range(1, len(nodesPerLayer))]
    v_bias = [np.zeros((nodesPerLayer[i], 1)) for i in range(1, len(nodesPerLayer))]

    num_batches = len(x_flatten_train) // batch_size

    for epoch in range(epochs):
        for batch in range(0,num_batches):
            start = batch * batch_size
            end = (batch + 1) * batch_size
            batch_x = x_flatten_train[start:end]
            batch_y = y_encoded[start:end]

            batch_w_delta = [np.zeros_like(w) for w in weights]
            batch_b_delta = [np.zeros_like(b) for b in bias]

            for j in range(len(batch_x)):
                A, B, C = forwardProp(batch_x[j], activationFunc, "crossEntropy", weights, bias)
                CurrWdelta, CurrBdelta = backProp(batch_y[j], C, B, weights, activationFunc, A)

                for k in range(len(CurrWdelta)):
                    batch_w_delta[k] += CurrWdelta[k]
                    batch_b_delta[k] += CurrBdelta[k]

            for k in range(len(batch_w_delta)):
                m_weights[k] = beta1 * m_weights[k] + (1 - beta1) * batch_w_delta[k]
                v_weights[k] = beta2 * v_weights[k] + (1 - beta2) * (batch_w_delta[k] ** 2)
                m_bias[k] = beta1 * m_bias[k] + (1 - beta1) * batch_b_delta[k]
                v_bias[k] = beta2 * v_bias[k] + (1 - beta2) * (batch_b_delta[k] ** 2)

                m_weights_hat = m_weights[k] / (1 - beta1 ** (epoch + 1))
                v_weights_hat = v_weights[k] / (1 - beta2 ** (epoch + 1))
                m_bias_hat = m_bias[k] / (1 - beta1 ** (epoch + 1))
                v_bias_hat = v_bias[k] / (1 - beta2 ** (epoch + 1))

                weights[k] =((1 - (lr * lambda_reg / batch_size)) * weights[k]) - ( lr * (beta1 * m_weights_hat + (1 - beta1) * batch_w_delta[k]) / (np.sqrt(v_weights_hat) + eps))
                bias[k] -= lr * (beta1 * m_bias_hat + (1 - beta1) * batch_b_delta[k]) / (np.sqrt(v_bias_hat) + eps)
        accuracy,loss = testModel(weights,bias,x_flatten_test,y_flatten_test,activationFunc)
        Valaccuracy,Valloss = testModel(weights,bias,x_val,y_val,activationFunc)
        wandb.log({"val_loss":Valloss,"val_accuracy":Valaccuracy,"loss":loss,"accuracy":accuracy,"epoch":epoch})

    return weights, bias

In [16]:
def gradient_descent(nodesPerLayer,x_flatten_train,y_encoded,activationFunc,epochs,lr):
  weights = list()
  bias = list()
  w = np.random.randn(nodesPerLayer[i],nodesPerLayer[i-1])*0.1
  b =  np.random.randn(nodesPerLayer[i],1)
  weights.append(w)
  bias.append(b)
  Wdelta = list()
  Bdelta = list()
  for i in range(0,epochs):
    Wdelta.clear()
    Bdelta.clear()
    for j in range(0,len(y_encoded)):
      A,B,C = forwardProp(x_flatten_train[j],activationFunc,"crossEntropy",weights,bias)
      CurrWdelta,CurrBdelta = backProp(y_encoded[j],C,B,weights,activationFunc,A)
      if( len(Wdelta) == 0):
        Wdelta =  copy.deepcopy(CurrWdelta)
        Bdelta = copy.deepcopy(CurrBdelta)
      else:
        for k in range(0,len(Wdelta)):
          Wdelta[k] = Wdelta[k] + CurrWdelta[k]
          Bdelta[k] = Bdelta[k] + CurrBdelta[k]
      if(j%1000 == 0):
        print(j/1000)
    for k in range(0,len(weights)):
      weights[k] = weights[k] - lr*Wdelta[k]
      bias[k] = bias[k] - lr*Bdelta[k]
  return weights,bias

In [17]:
def executeTraining(config,x_train,y_train,x_flatten_test,y_flatten_test,x_val,y_val):
  FinalWeights = list()
  FinalBias = list()
  beta1 = 0.89
  beta2 = 0.95
  eps = 1e-6
  batch_size = config.batch_size
  layers = config.number_hidden
  lr = config.learning_rate
  epochs = config.epochs
  activationFunc = config.activation
  optimizer = config.optimizer
  weightInit = config.weight_init
  lambda_reg = config.weight_decay
  nodesPerLayer = list()
  nodesPerLayer.append(784)
  for i in range(0,layers):
    nodesPerLayer.append(config.hidden_inputsize)
  nodesPerLayer.append(10)

  gamma = 0.6
  betarms = 0.6
  if(optimizer == "gradient_descent"):
    FinalWeights, FinalBias = gradient_descent(nodesPerLayer,x_train,y_train,activationFunc,epochs,lr)
  elif(optimizer == "sgd"):
    FinalWeights, FinalBias = stochastic_gradient_descent(nodesPerLayer,x_train,y_train,batch_size,activationFunc,epochs,lr,x_flatten_test,y_flatten_test,x_val,y_val,weightInit,lambda_reg)
  elif(optimizer == "momentumGD"):
    FinalWeights, FinalBias = momentum_gradient_descent(nodesPerLayer,x_train,y_train,gamma,batch_size,activationFunc,epochs,lr,x_flatten_test,y_flatten_test,x_val,y_val,weightInit,lambda_reg)
  elif(optimizer == "nesterov"):
    FinalWeights, FinalBias = nesterov_gradient_descent(nodesPerLayer,x_train,y_train,gamma,batch_size,activationFunc,epochs,lr,x_flatten_test,y_flatten_test,x_val,y_val,weightInit,lambda_reg)
  elif(optimizer == "rmsprop"):
    FinalWeights, FinalBias =rmsprop(nodesPerLayer,x_train,y_train,betarms,eps,epochs,batch_size,lr,activationFunc,x_flatten_test,y_flatten_test,x_val,y_val,weightInit,lambda_reg)
  elif(optimizer == "adam"):
    FinalWeights, FinalBias = adam(nodesPerLayer, x_train, y_train, beta1, beta2, eps, batch_size,lr,activationFunc,epochs,x_flatten_test,y_flatten_test,x_val,y_val,weightInit,lambda_reg)
  elif(optimizer == "nadam"):
    FinalWeights, FinalBias = nadam(nodesPerLayer, x_train, y_train, beta1, beta2, eps, batch_size,lr,activationFunc,epochs,x_flatten_test,y_flatten_test,x_val,y_val,weightInit,lambda_reg)
  return FinalWeights,FinalBias

In [18]:
def testModel(weights,bias,x_test,y_test,activationFun):
  count = 0
  loss = 0.0

  for i in range(0,x_test.shape[0]):
    A,B,C = forwardProp(x_test[i],activationFun,"crossEntropy",weights,bias)
    if( y_test[i]==np.argmax(C)):
      count+=1
      loss += -np.log(C)[y_test[i]][0]
  loss /= y_test.shape[0]
  acc = (count/y_test.shape[0])
  return acc,loss


In [19]:

# load dataset
def load_data():

  (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
  x_train, y_train = shuffle(x_train, y_train)
  val_size = int(x_train.shape[0] * 0.1)

  y_val = y_train[:val_size]
  y_new_train = y_train[val_size:]

  x_flatten_train = x_train.reshape(x_train.shape[0],x_train.shape[1]*x_train.shape[2],1)
  x_flatten_train = normalize_data(x_flatten_train)

  y_encoded = np.zeros((y_new_train.shape[0], 10))
  y_encoded[np.arange(y_new_train.shape[0]), y_new_train] = 1
  y_new_train = y_encoded.reshape(y_new_train.shape[0],10,1)


  x_flatten_test = x_test.reshape(x_test.shape[0],x_test.shape[1]*x_test.shape[2],1)
  x_flatten_test = normalize_data(x_flatten_test)


  x_val = x_flatten_train[:val_size]


  x_new_train = x_flatten_train[val_size:]


  return x_new_train, y_new_train,x_val,y_val,x_flatten_test,y_test


In [20]:
def executeTraining1(config,x_train,y_train,x_flatten_test,y_flatten_test,x_val,y_val):
  FinalWeights = list()
  FinalBias = list()
  beta1 = 0.89
  beta2 = 0.95
  eps = 1e-6
  batch_size = config['parameters']['batch_size']['values'][-1]
  layers = config['parameters']['number_hidden']['values'][0]
  lr = config['parameters']['learning_rate']['values'][0]
  epochs = config['parameters']['epochs']['values'][0]
  activationFunc = config['parameters']['activation']['values'][1]
  optimizer = config['parameters']['optimizer']['values'][-1]
  weightInit = config['parameters']['weight_init']['values'][-1]
  lambda_reg = config['parameters']['weight_decay']['values'][1]
  nodesPerLayer = list()
  nodesPerLayer.append(784)
  for i in range(0,layers):
    nodesPerLayer.append(config['parameters']['hidden_inputsize']['values'][-1])
  nodesPerLayer.append(10)

  gamma = 0.6
  betarms = 0.6
  if(optimizer == "gradient_descent"):
    FinalWeights, FinalBias = gradient_descent(nodesPerLayer,x_train,y_train,activationFunc,epochs,lr)
  elif(optimizer == "sgd"):
    FinalWeights, FinalBias = stochastic_gradient_descent(nodesPerLayer,x_train,y_train,batch_size,activationFunc,epochs,lr,x_flatten_test,y_flatten_test,x_val,y_val,weightInit,lambda_reg)
  elif(optimizer == "momentumGD"):
    FinalWeights, FinalBias = momentum_gradient_descent(nodesPerLayer,x_train,y_train,gamma,batch_size,activationFunc,epochs,lr,x_flatten_test,y_flatten_test,x_val,y_val,weightInit,lambda_reg)
  elif(optimizer == "nesterov"):
    FinalWeights, FinalBias = nesterov_gradient_descent(nodesPerLayer,x_train,y_train,gamma,batch_size,activationFunc,epochs,lr,x_flatten_test,y_flatten_test,x_val,y_val,weightInit,lambda_reg)
  elif(optimizer == "rmsprop"):
    FinalWeights, FinalBias =rmsprop(nodesPerLayer,x_train,y_train,betarms,eps,epochs,batch_size,lr,activationFunc,x_flatten_test,y_flatten_test,x_val,y_val,weightInit,lambda_reg)
  elif(optimizer == "adam"):
    FinalWeights, FinalBias = adam(nodesPerLayer, x_train, y_train, beta1, beta2, eps, batch_size,lr,activationFunc,epochs,x_flatten_test,y_flatten_test,x_val,y_val,weightInit,lambda_reg)
  elif(optimizer == "nadam"):
    FinalWeights, FinalBias = nadam(nodesPerLayer, x_train, y_train, beta1, beta2, eps, batch_size,lr,activationFunc,epochs,x_flatten_test,y_flatten_test,x_val,y_val,weightInit,lambda_reg)
  return FinalWeights,FinalBias

In [21]:
sweep_config = {
    'name':"my-sweep",
    'method': 'bayes',
    'metric': {
      'name': 'accuracy',
      'goal': 'maximize'
    },

    'parameters': {
        'epochs': {
            'values': [5, 10] #number of epochs
        },
        'number_hidden': {
            'values': [3, 4, 5] #number of hidden layers
        },
        'hidden_inputsize': {
            'values':[32, 64, 128] #size of every hidden layer
        },
        'weight_decay': {
            'values':[0, 0.0005,  0.5] #L2 regularisation
        },
        'learning_rate': {
            'values': [1e-3, 1e-4]
        },
        'optimizer': {
            'values': ['momentumGD', 'nesterov', 'rmsprop', 'adam', 'nadam','sgd']
        },
        'batch_size' : {
            'values':[16, 32, 64]
        },
        'weight_init': {
            'values':['random','xavier']
        },
        'activation': {
            'values': ['sigmoid','tanh','relu']
        }

        }
}

sweep_id = wandb.sweep(sweep_config, entity="CS23M028", project="Assignment 1")

Create sweep with ID: x5jc8xps
Sweep URL: https://wandb.ai/cs23m028/Assignment%201/sweeps/x5jc8xps


In [ ]:
x_new_train, y_new_train,x_val,y_val,x_flatten_test,y_flatten_test = load_data()
trainedWeights,trainedBias = executeTraining1(sweep_config,x_new_train,y_new_train,x_flatten_test,y_flatten_test,x_val,y_val)

batch: 0
batch: 5400
batch: 10800
batch: 16200
batch: 21600
batch: 27000
batch: 32400
batch: 37800
batch: 43200
batch: 48600
{'val_loss': 0.7142123826810086, 'val_accuracy': 0.6998333333333333, 'loss': 0.6870950239729352, 'accuracy': 0.6782, 'epoch': 0}
batch: 0
batch: 5400
batch: 10800
batch: 16200
batch: 21600
batch: 27000
batch: 32400
batch: 37800
batch: 43200
batch: 48600
{'val_loss': 0.5152079192871214, 'val_accuracy': 0.737, 'loss': 0.4891780796878548, 'accuracy': 0.7126, 'epoch': 1}
batch: 0
batch: 5400
batch: 10800
batch: 16200
batch: 21600
batch: 27000
batch: 32400
batch: 37800
batch: 43200
batch: 48600
{'val_loss': 0.4212484274779807, 'val_accuracy': 0.7571666666666667, 'loss': 0.40324468716569556, 'accuracy': 0.7371, 'epoch': 2}
batch: 0
batch: 5400
batch: 10800
batch: 16200
batch: 21600


In [ ]:
def fitModel():
  with wandb.init() as run:
    config = wandb.config
    wandb.run.name = "hidden_" + str(config.hidden_inputsize)+"_batchSize_"+str(config.batch_size)+"_acc_"+ config.activation
    np.random.seed(1)

    x_new_train, y_new_train,x_val,y_val,x_flatten_test,y_flatten_test = load_data()
    trainedWeights,trainedBias = executeTraining(config,x_new_train,y_new_train,x_flatten_test,y_flatten_test,x_val,y_val)

wandb.agent(sweep_id,fitModel,entity="CS23M028", project="Assignment 1")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
Exception in thread NetStatThr:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
Exception in thread IntMsgThr:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_run.py", line 300, in check_internal_messages
    self._loop_check_status(
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_run.py", line 224, in _loop_check_status
    local_handle = request()
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/interface/interface.py", line 803, in deliver_internal_messages
        return self._deliver_internal_messages(internal_m

4422102/4422102 [==============================] - 0s 0us/step
batch :0
batch :5400
batch :10800
batch :16200
batch :21600
batch :27000
batch :32400
batch :37800
batch :43200
batch :48600
{'val_loss': 0.3412825402433474, 'val_accuracy': 0.17916666666666667, 'loss': 0.35948971709925065, 'accuracy': 0.1891, 'epoch': 0}
batch :0
batch :5400
batch :10800
batch :16200
batch :21600
batch :27000
batch :32400
batch :37800
batch :43200
batch :48600
{'val_loss': 0.5835106336302095, 'val_accuracy': 0.42233333333333334, 'loss': 0.5950381876919387, 'accuracy': 0.4306, 'epoch': 1}
batch :0
batch :5400
batch :10800
batch :16200
batch :21600
batch :27000
batch :32400
batch :37800
batch :43200
batch :48600
{'val_loss': 0.5937274932946812, 'val_accuracy': 0.646, 'loss': 0.5811077340586168, 'accuracy': 0.6371, 'epoch': 2}
batch :0
batch :5400
batch :10800
batch :16200
batch :21600
batch :27000
batch :32400
batch :37800
batch :43200
batch :48600
{'val_loss': 0.42675707480643493, 'val_accuracy': 0.69033333

accuracy,▁▄▇██
epoch,▁▃▅▆█
loss,▂██▃▁
val_accuracy,▁▄▇██
val_loss,▁██▃▁
accuracy,0.7131
epoch,4
loss,0.33815
val_accuracy,0.71367
val_loss,0.34203


wandb: Agent Starting Run: 6x2vgwp3 with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 16
wandb: 	epochs: 5
wandb: 	hidden_inputsize: 64
wandb: 	learning_rate: 0.0001
wandb: 	number_hidden: 3
wandb: 	optimizer: sgd
wandb: 	weight_decay: 0
wandb: 	weight_init: xavier
wandb: Currently logged in as: cs23m028. Use `wandb login --relogin` to force relogin


batch: 0
batch: 5400
batch: 10800
batch: 16200
batch: 21600
batch: 27000
batch: 32400
batch: 37800
batch: 43200
batch: 48600
{'val_loss': 0.09042104201330622, 'val_accuracy': 0.04733333333333333, 'loss': 0.09247409168931366, 'accuracy': 0.0484, 'epoch': 0}
batch: 0
batch: 5400
batch: 10800
batch: 16200
batch: 21600
batch: 27000
batch: 32400
batch: 37800
batch: 43200
batch: 48600
{'val_loss': 0.17041786927373664, 'val_accuracy': 0.082, 'loss': 0.18706738766213984, 'accuracy': 0.09, 'epoch': 1}
batch: 0
batch: 5400
batch: 10800
batch: 16200
batch: 21600
batch: 27000
batch: 32400
batch: 37800
batch: 43200
batch: 48600
{'val_loss': 0.17899642603604626, 'val_accuracy': 0.08233333333333333, 'loss': 0.1974229980021158, 'accuracy': 0.0908, 'epoch': 2}
batch: 0
batch: 5400
batch: 10800
batch: 16200
batch: 21600
batch: 27000
batch: 32400
batch: 37800
batch: 43200
batch: 48600
{'val_loss': 0.14888876395333067, 'val_accuracy': 0.06683333333333333, 'loss': 0.16130006928143292, 'accuracy': 0.0724, '

...

accuracy,█▄▃▁▁
epoch,▁▃▅▆█
loss,▁▆█▆▂
val_accuracy,█▅▄▁▃
val_loss,▁▆█▆▂
accuracy,0.814
epoch,4
loss,0.70888
val_accuracy,0.82233
val_loss,0.7188


wandb: Agent Starting Run: 8wz5ouez with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 64
wandb: 	epochs: 5
wandb: 	hidden_inputsize: 32
wandb: 	learning_rate: 0.0001
wandb: 	number_hidden: 4
wandb: 	optimizer: nadam
wandb: 	weight_decay: 0
wandb: 	weight_init: random


accuracy,▁▁▃▇█
epoch,▁▃▅▆█
loss,█▁▅█▅
val_accuracy,▁▁▃▇█
val_loss,█▁▄▇▅
accuracy,0.317
epoch,4
loss,0.38604
val_accuracy,0.31983
val_loss,0.39724


wandb: Agent Starting Run: toua5rxo with config:
wandb: 	activation: relu
wandb: 	batch_size: 16
wandb: 	epochs: 5
wandb: 	hidden_inputsize: 64
wandb: 	learning_rate: 0.0001
wandb: 	number_hidden: 4
wandb: 	optimizer: momentumGD
wandb: 	weight_decay: 0.0005
wandb: 	weight_init: xavier


batch :0
batch :5400
batch :10800
batch :16200
batch :21600
batch :27000
batch :32400
batch :37800
batch :43200
batch :48600
{'val_loss': 0.3759968504842956, 'val_accuracy': 0.2055, 'loss': 0.3661192231215538, 'accuracy': 0.1991, 'epoch': 0}
batch :0


In [ ]:
classes = set(y_train)
names = ["T-shirt/top","Trouser/pants","Pullover shirt","Dress","Coat","Sandal","Shirt","Sneaker","Bag","Ankle boot"]

variousSamples = list()
for i in classes:
  ind = np.where(y_train == i)[0][0]
  variousSamples.append(wandb.Image(x_train[ind],caption = names[i]))

wandb.log({"examples": variousSamples})  #logs images to the wandb panel

26421880/26421880 [==============================] - 0s 0us/step


BrokenPipeError: [Errno 32] Broken pipe

4422102/4422102 [==============================] - 0s 0us/step


NameError: name 'y_train' is not defined

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7ffaae2773a0>> (for post_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

Traceback (most recent call last):
  File "<ipython-input-20-a3a5ebfef912>", line 7, in fitModel
    x_new_train, y_new_train,x_val,y_val,x_flatten_test,y_flatten_test = load_data()
  File "<ipython-input-18-5803c13dc010>", line 6, in load_data
    y_encoded = np.zeros((y_train.shape[0], max(classes) + 1))
NameError: name 'classes' is not defined
